In [2]:
import json
import requests
import pandas as pd
from riotwatcher import LolWatcher, ApiError

with open('data/api-key.txt', 'r') as api:
    API_KEY = api.read()
lol_watcher = LolWatcher(API_KEY)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
}

keyid = pd.read_csv("data/champion-key-ids.csv")

In [3]:
def get_matches(region, summ_name, champion_key="", index=""):
    summoner_obj = lol_watcher.summoner.by_name(region, summ_name)
    if champion_key != "":
        champion_key = "".join(["?champion=",str(champion_key)])
    if index!="":
        index = "".join(["&beginIndex=", str(index)])
    if region == "kr":
        summ_url = "".join(["https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/",
                           summoner_obj["accountId"], champion_key, index])
    elif region == "na":
        summ_url = "".join(["https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/",
                           summoner_obj["accountId"], champion_key, index])
    summ_response = requests.get(summ_url,headers=HEADERS)
    summ_json = json.loads(summ_response.text)
    try:
        return pd.DataFrame(summ_json["matches"])
    except KeyError as e:
        print(summ_response)
        raise e

In [4]:
get_matches("kr", "Gen G Clid", 64)

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4768817195,64,420,13,1604762698657,DUO_SUPPORT,NONE
1,KR,4768709088,64,420,13,1604760613378,NONE,JUNGLE
2,KR,4766414121,64,420,13,1604685379075,NONE,JUNGLE
3,KR,4766279291,64,420,13,1604682892465,NONE,JUNGLE
4,KR,4766343350,64,420,13,1604680882287,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
95,KR,4476545133,64,420,13,1593351144270,NONE,JUNGLE
96,KR,4471816100,64,420,13,1593180502976,DUO,NONE
97,KR,4470325921,64,420,13,1593108680782,NONE,JUNGLE
98,KR,4470371078,64,420,13,1593105168010,NONE,JUNGLE


In [5]:
get_matches("kr", "Gen G Clid", 64, 100)

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4468416958,64,420,13,1593013312548,NONE,JUNGLE
1,KR,4466385623,64,420,13,1592929587778,NONE,JUNGLE
2,KR,4464408055,64,420,13,1592847467928,NONE,JUNGLE
3,KR,4456274649,64,420,13,1592502156071,NONE,JUNGLE
4,KR,4456185422,64,420,13,1592496514604,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
95,KR,4143221698,64,420,13,1581324540803,NONE,JUNGLE
96,KR,4132407280,64,420,13,1580886115379,NONE,JUNGLE
97,KR,4129476635,64,420,13,1580752020523,NONE,JUNGLE
98,KR,4129450549,64,420,13,1580748479918,NONE,JUNGLE


In [6]:
def get_all_matches(region, summ_name, champion_key):
    index = 0
    match_batches = []
    while True:
        current = get_matches(region, summ_name, champion_key, index)
        if len(current)==0:
            break
        match_batches.append(current)
        index+=100
    return pd.concat(match_batches)

In [7]:
matches = get_all_matches("kr", "Gen G Clid", 64)

In [8]:
matches

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,KR,4768817195,64,420,13,1604762698657,DUO_SUPPORT,NONE
1,KR,4768709088,64,420,13,1604760613378,NONE,JUNGLE
2,KR,4766414121,64,420,13,1604685379075,NONE,JUNGLE
3,KR,4766279291,64,420,13,1604682892465,NONE,JUNGLE
4,KR,4766343350,64,420,13,1604680882287,NONE,JUNGLE
...,...,...,...,...,...,...,...,...
57,KR,3420476831,64,420,12,1542113321062,NONE,JUNGLE
58,KR,3419240466,64,420,11,1542012496649,DUO_SUPPORT,NONE
59,KR,3419165068,64,420,11,1542010014885,DUO_SUPPORT,NONE
60,KR,3419108414,64,420,11,1542007670086,NONE,JUNGLE


In [9]:
matches = matches[matches["lane"]=="JUNGLE"]

In [10]:
matches.to_csv("data/clid-matches.csv", index=False)